In [1]:
import pandas as pd
import os, fnmatch
import xlrd
import difflib
import re
                                   

In [9]:
# find path to all the dm.xpt files given an application name
def find_dm_file (app_no):
    path = r"\\cdsesub1\evsprod\\"+app_no
    #print path

    result = []
    
    for root, dirs, files in os.walk(path):
        for direct in dirs:
            dirpath = os.path.join(root, direct)
            for name in os.listdir(dirpath):
                if fnmatch.fnmatch(name, "dm.xpt"):
                    result.append(os.path.join(root, direct, name))
         
        
    return result
    

In [10]:
list_of_dm_files = find_dm_file("BLA125504")

In [11]:
list_of_dm_files

['\\\\cdsesub1\\evsprod\\\\BLA125504\\0000\\m5\\datasets\\ain457a2302\\tabulations\\sdtm\\dm.xpt',
 '\\\\cdsesub1\\evsprod\\\\BLA125504\\0000\\m5\\datasets\\ain457a2303\\tabulations\\sdtm\\dm.xpt',
 '\\\\cdsesub1\\evsprod\\\\BLA125504\\0000\\m5\\datasets\\ain457a2304\\tabulations\\sdtm\\dm.xpt',
 '\\\\cdsesub1\\evsprod\\\\BLA125504\\0000\\m5\\datasets\\ain457a2307\\tabulations\\sdtm\\dm.xpt',
 '\\\\cdsesub1\\evsprod\\\\BLA125504\\0000\\m5\\datasets\\ain457a2308\\tabulations\\sdtm\\dm.xpt',
 '\\\\cdsesub1\\evsprod\\\\BLA125504\\0000\\m5\\datasets\\ain457a2309\\tabulations\\sdtm\\dm.xpt',
 '\\\\cdsesub1\\evsprod\\\\BLA125504\\0060\\m5\\datasets\\ain457f2305\\tabulations\\sdtm\\dm.xpt',
 '\\\\cdsesub1\\evsprod\\\\BLA125504\\0060\\m5\\datasets\\ain457f2306\\tabulations\\sdtm\\dm.xpt',
 '\\\\cdsesub1\\evsprod\\\\BLA125504\\0060\\m5\\datasets\\ain457f2310\\tabulations\\sdtm\\dm.xpt',
 '\\\\cdsesub1\\evsprod\\\\BLA125504\\0060\\m5\\datasets\\ain457f2312\\tabulations\\sdtm\\dm.xpt',
 '\\\\cdse

In [8]:
biggest_dm_file = biggest_file(list_of_dm_files)

In [9]:
biggest_dm_file

'\\\\cdsesub1\\evsprod\\\\BLA761029\\0000\\m5\\datasets\\205ms301\\tabulations\\sdtm\\dm.xpt'

In [4]:
# Importing data into pandas data frame
df_dac = pd.read_sas(r'\\cdsesub1\\evsprod\\BLA761029\\0000\\m5\\datasets\\205ms201\\tabulations\\sdtm\\dm.xpt')

In [53]:
len(df_dac)

621

In [2]:
data_elements_file =  pd.ExcelFile("C:\Users\Md.Shamsuzzaman\Documents\OCS\AIH\data_elements_ver2.xlsx")

In [3]:
# View the excel file's sheet names
data_elements_file.sheet_names

[u'Domains', u'LB', u'LB CodeList', u'DM', u'DM CodeList', u'Sheet1', u'AE']

In [4]:
# Load the xls file's Sheet1 as a dataframe
df = data_elements_file.parse('DM')
df.head()

,Variable Name,Variable Label,Type,"Controlled Terms, Codelist or Format",Role,CDISC Notes,Core,Remark
0,APPLICID,Application Identifier,Char,NaN,Identifier,Unique identifier for a study.,Req,User definded
1,STUDYID,Study Identifier,Char,NaN,Identifier,Unique identifier for a study.,Req,NaN
2,DOMAIN,Domain Abbreviation,Char,DM,Identifier,Two-character abbreviation for the domain.,Req,NaN
3,USUBJID,Unique Subject Identifier,Char,NaN,Identifier,Identifier used to uniquely identify a subject...,Req,NaN
4,RFSTDTC,Subject Reference Start Date/Time,Char,ISO 8601,Record Qualifier,Reference Start Date/time for the subject in I...,Exp,NaN


In [8]:
# not collecting all the var at this moment - like ACTARM
df = df.iloc[:17,:]

In [9]:
# Data elements needs to be extracted from DM domains
DM_data_elements = list(df['Variable Name'])

In [10]:
# Not all 17 are present in Daclizumab DM domain. Hence, create an empty df with variables needed first

dm_df = pd.DataFrame(index = range(len(df_dac)), columns = DM_data_elements)

In [11]:
# find common col var 
common_var = list(set(DM_data_elements).intersection(list(df_dac)))

In [12]:
#how many are common
len(common_var)

11

In [13]:
# find the vars which are missing in dm_df
list(set(DM_data_elements) - set(common_var))

[u'RFXENDTC', u'RFXSTDTC', u'DMDTC', u'DMDY', u'ETHNIC', u'APPLICID']

In [54]:
# fill up the dm_df with daclizumab data
dm_df['APPLICID'] = 'BLA761029'
dm_df[common_var] = df_dac[common_var]
# keep these empty
#dm_df['RFXSTDTC'] = df_dac['RFSTDTC']
#dm_df['RFXENDTC'] = df_dac['RFENDTC']

In [55]:
# Check the code values
set(list(dm_df['RACE']))

{'ASIAN', 'WHITE'}

In [ ]:
# change the no reported value to other ------------- Lets not convert it
#dm_df.loc[dm_df['RACE'] =='NOT REPORTED DUE TO CONFIDENTIALITY REGULATIONS','RACE'] = 'OTHER'

In [56]:
dm_df.head()

,APPLICID,STUDYID,DOMAIN,USUBJID,RFSTDTC,RFENDTC,RFXSTDTC,RFXENDTC,AGE,SEX,RACE,ETHNIC,ARMCD,ARM,COUNTRY,DMDTC,DMDY
0,BLA761029,205MS201,DM,205MS201/100-001,2010-02-25T11:50,2011-01-25T11:30,NaN,NaN,36.0,F,ASIAN,NaN,1,150 mg DAC HYP,IND,NaN,NaN
1,BLA761029,205MS201,DM,205MS201/100-002,2010-03-22T12:35,2011-02-22T11:15,NaN,NaN,39.0,F,ASIAN,NaN,2,300 mg DAC HYP,IND,NaN,NaN
2,BLA761029,205MS201,DM,205MS201/100-003,2010-04-28T13:30,2011-04-05T14:20,NaN,NaN,42.0,M,ASIAN,NaN,2,300 mg DAC HYP,IND,NaN,NaN
3,BLA761029,205MS201,DM,205MS201/100-004,2010-04-28T12:15,2011-03-30T10:45,NaN,NaN,20.0,M,ASIAN,NaN,1,150 mg DAC HYP,IND,NaN,NaN
4,BLA761029,205MS201,DM,205MS201/101-001,2010-03-04T13:45,2011-02-05T12:00,NaN,NaN,24.0,F,ASIAN,NaN,2,300 mg DAC HYP,IND,NaN,NaN


In [57]:
dm_df.to_csv("C:/Users/Md.Shamsuzzaman/Documents/OCS/AIH/Daclizumab/205ms201_DM2.csv", index = False)

In [ ]:
len(dm_df)

In [ ]:
dm1 = dm1.append(dm_df)
len(dm1)

In [ ]:
dm1.to_csv("C:/Users/Md.Shamsuzzaman/Documents/OCS/AIH/Daclizumab/BLA761029_DM.csv", index = False)

In [2]:
# Importing Phase 3 lB data into pandas data frame
df_dac = pd.read_sas(r'\\cdsesub1\\evsprod\\BLA761029\\0014\\m5\\datasets\\205ms301\\tabulations\\sdtm\\lb.xpt')

In [3]:
df_dac.head()

,STUDYID,DOMAIN,USUBJID,LBSEQ,LBREFID,LBTESTCD,LBTEST,LBCAT,LBSCAT,LBORRES,...,LBSTAT,LBREASND,LBNAM,LBBLFL,LBTOXGR,VISITNUM,VISIT,EPOCH,LBDTC,LBDY
0,205MS301,LB,205MS301/100-004,1.0,L983846,ALP,Alkaline Phosphatase,BLOOD CHEMISTRY,,52,...,,,CCLS_CO,,0,10.0,SCREENING,SCREENING,2011-12-06T03:25,-14.0
1,205MS301,LB,205MS301/100-004,2.0,L983857,ALP,Alkaline Phosphatase,BLOOD CHEMISTRY,,52,...,,,CCLS_CO,Y,0,20.0,WEEK 0,SCREENING,2011-12-20T13:29,1.0
2,205MS301,LB,205MS301/100-004,3.0,Q437169,ALP,Alkaline Phosphatase,BLOOD CHEMISTRY,,51,...,,,CCLS_CO,,0,30.0,WEEK 4,TREATMENT,2012-01-16T09:34,28.0
3,205MS301,LB,205MS301/100-004,4.0,R799628,ALP,Alkaline Phosphatase,BLOOD CHEMISTRY,,47,...,,,CCLS_CO,,0,40.0,WEEK 8,TREATMENT,2012-02-17T10:08,60.0
4,205MS301,LB,205MS301/100-004,5.0,R799617,ALP,Alkaline Phosphatase,BLOOD CHEMISTRY,,51,...,,,CCLS_CO,,0,50.0,WEEK 12,TREATMENT,2012-03-16T11:50,88.0


In [5]:
# Load the xls file's Sheet1 as a dataframe
df = data_elements_file.parse('LB')
df.head()

,Variable Name,Variable Label,Type,"Controlled Terms, Codelist or Format",Role,CDISC Notes,Core,Remark
0,APPLICID,Application Identifier,Char,NaN,Identifier,Unique identifier for a study.,Req,User definded
1,STUDYID,Study Identifier,Char,NaN,Identifier,Unique identifier for a study.,Req,NaN
2,DOMAIN,Domain Abbreviation,Char,LB,Identifier,Two-character abbreviation for the domain.,Req,NaN
3,USUBJID,Unique Subject Identifier,Char,NaN,Identifier,Identifier used to uniquely identify a subject...,Req,NaN
4,LBSEQ,Sequence Number,Num,NaN,Identifier,Sequence Number given to ensure uniqueness of ...,Req,NaN


In [6]:
# not collecting all the var at this moment - like ACTARM
df = df.iloc[:25,:]
df.tail(3)

,Variable Name,Variable Label,Type,"Controlled Terms, Codelist or Format",Role,CDISC Notes,Core,Remark
22,LBDTC,Date/Time of Specimen\nCollection,Char,ISO 8601,Timing,NaN,Exp,NaN
23,LBENDTC,End Date/Time of\nSpecimen Collection,Char,ISO 8601,Timing,NaN,Perm,NaN
24,LBDY,Study Day of Specimen\nCollection,Num,NaN,Timing,"1. Study day of specimen collection, measured ...",Perm,NaN


In [7]:
# Data elements needs to be extracted from LB domains
LB_data_elements = list(df['Variable Name'])

In [8]:
# Not all are present in Daclizumab LB domain. Hence, create an empty df with variables needed first

lb_df = pd.DataFrame(index = range(len(df_dac)), columns = LB_data_elements)

NameError: name 'df_dac' is not defined

In [11]:
# find common col var 
common_var = list(set(LB_data_elements).intersection(list(df_dac)))

In [12]:
len(common_var)

20

In [13]:
# find the vars which are present in lb_df and missing in df_dac
list(set(LB_data_elements) - set(common_var))

[u'APPLICID', u'LBTOX', u'LBENDTC', u'LBFAST', u'LBSTNRC']

In [67]:
# find the vars which are missing in df_dac
list(set(list(df_dac)) - set(common_var))

[u'LBORRES',
 u'LBORRESU',
 u'LBORNRHI',
 u'LBSCAT',
 u'EPOCH',
 u'LBREFID',
 u'LBNAM',
 u'LBORNRLO',
 u'LBNRIND']

In [14]:
# fill up the dm_df with daclizumab data
lb_df['APPLICID'] = 'BLA761029'
lb_df[common_var] = df_dac[common_var]

In [15]:
# Check the LBTEST code values
dac_cd = set(list(lb_df['LBTESTCD']))

In [24]:
# Load the xls file's Sheet1 as a dataframe
df_code = data_elements_file.parse('LB CodeList')
df_code = df_code.iloc[:125,:]
df_code.tail(5)

,LBTESTCD,LBTEST,LBTESTDSC,LBCAT,LBSTRESU
120,HCVGTYPE,HCV Genotype,NaN,MICROBIOLOGY,NaN
121,HCRTQE,HCV RNA Taq 2.0-Exp,NaN,HEMATOLOGY,NaN
122,HEVRNA,Hepatitis E Virus RNA,NaN,HEMATOLOGY,NaN
123,HIGMC,Herpes Sim.Virus 1/2 IgM IFA Confirm,NaN,SPECIAL CHEMISTRY,NaN
124,HIGMIN,Herpes Sim.Virus 1/2 IgM Interpretation,NaN,MICROBIOLOGY,NaN


In [25]:
# code elements needs to be extracted from LB domains
req_code = list(df_code['LBTESTCD'])

In [18]:
# find common col var 
common_var = list(set(req_code).intersection(list(dac_cd)))

In [19]:
len(common_var)

62

In [20]:
# find the code which are missing in dac_cd
unique_req_cd = list(set(req_code) - set(common_var))

In [39]:
df_dac[df_dac.LBTESTCD == "SCSPMS1P"].iloc[:,5:20]

,LBTESTCD,LBTEST,LBCAT,LBSCAT,LBORRES,LBORRESU,LBORNRLO,LBORNRHI,LBSTRESC,LBSTRESN,LBSTRESU,LBSTNRLO,LBSTNRHI,LBNRIND,LBSTAT
3366,SCSPMS1P,SPE-M-spike 1 -Percent,SPECIAL CHEMISTRY,,0.0,%,0,9999,0,5.397605e-79,%,5.397605e-79,9999.0,N,
6774,SCSPMS1P,SPE-M-spike 1 -Percent,SPECIAL CHEMISTRY,,0.0,%,0,9999,0,5.397605e-79,%,5.397605e-79,9999.0,N,
22212,SCSPMS1P,SPE-M-spike 1 -Percent,SPECIAL CHEMISTRY,,0.0,%,0,9999,0,5.397605e-79,%,5.397605e-79,9999.0,N,
22899,SCSPMS1P,SPE-M-spike 1 -Percent,SPECIAL CHEMISTRY,,0.0,%,0,9999,0,5.397605e-79,%,5.397605e-79,9999.0,N,
24485,SCSPMS1P,SPE-M-spike 1 -Percent,SPECIAL CHEMISTRY,,0.0,%,0,9999,0,5.397605e-79,%,5.397605e-79,9999.0,N,
27393,SCSPMS1P,SPE-M-spike 1 -Percent,SPECIAL CHEMISTRY,,0.0,%,0,9999,0,5.397605e-79,%,5.397605e-79,9999.0,N,
62443,SCSPMS1P,SPE-M-spike 1 -Percent,SPECIAL CHEMISTRY,,0.0,%,0,9999,0,5.397605e-79,%,5.397605e-79,9999.0,N,
62444,SCSPMS1P,SPE-M-spike 1 -Percent,SPECIAL CHEMISTRY,,,,0,9999,,NaN,,5.397605e-79,9999.0,,NOT DONE
63086,SCSPMS1P,SPE-M-spike 1 -Percent,SPECIAL CHEMISTRY,,0.0,%,0,9999,0,5.397605e-79,%,5.397605e-79,9999.0,N,
78592,SCSPMS1P,SPE-M-spike 1 -Percent,SPECIAL CHEMISTRY,,,,0,9999,,NaN,,5.397605e-79,9999.0,,NOT DONE


In [25]:
# find the code which are unique to dac_code
unique_dac_cd = list(set(dac_cd) - set(common_var))

In [29]:
unique_dac_cd

['KETONES',
 'NEUTLE',
 'SCSPMSR1',
 'BACT',
 'SCSPMSR2',
 'FATBODOV',
 'CYTRPHOS',
 'CYCAOXA',
 'SODIUM',
 'GLOBG',
 'MONO',
 'GLOBBPT',
 'GLOBB',
 'PHENTHZ',
 'CYCACAR',
 'HS12GMAI',
 'CSWAX',
 'GLOBA2PT',
 'EPITUCE',
 'HEVVLD',
 'GLOBA1PT',
 'RBCCLMP',
 'GTTYIELD',
 'NEUTBLE',
 'GTYIELD1',
 'GTYIELD2',
 'EPITCE',
 'NEUTB',
 'CYCAPHOS',
 'SLAGAB',
 'EPISQCE',
 'PB19VLD',
 'METHQLDN',
 'EBCIGMAG',
 'LYMATLE',
 'METAMYLE',
 'NEUTSGLE',
 'GTA260_1',
 'COLOR',
 'CYCYSTIN',
 'BUN',
 'MYCY',
 'WBCCLMP',
 'PH',
 'EOSLE',
 'K',
 'SCSPMS2P',
 'HCG',
 'GLOBGPT',
 'LYMLE',
 'GTCONCN1',
 'GTCONCN2',
 'MCHC',
 'SCVZIGMT',
 'METAMY',
 'GTPURITY',
 'MUCTHR',
 'RBCMORPH',
 'NEUT',
 'EBN1IGAB',
 'MONOLE',
 'YEASTBUD',
 'CSHYAL',
 'CYTYRO',
 'CYURIAC',
 'HAABIGM',
 'APAP',
 'LEUKASE',
 'VZVINTP',
 'CYAMORPH',
 'CSGRAN',
 'CYAMMBIU',
 'MYCYLE',
 'BASOLE',
 'ALBPROT',
 'SCSPMSQ1',
 'SCSPMSQ2',
 'GTVOL2',
 'YEASTHYP',
 'GTVOL1',
 'APPEAR',
 'CYBILI',
 'SPGRAV',
 'SPERM',
 'GTHVOL',
 'GLOBA2',
 'GLOBA1',


In [22]:
# Drop duplicate rows from df_dac
df_dac_unique = df_dac.drop_duplicates(['LBTESTCD'], keep = 'first')

In [23]:
# req_code df with unique req code 
unique_df_code = df_code[df_code.LBTESTCD.isin(unique_req_cd)]

In [26]:
#dac_df with unique dac_code 
df_dac_unique = df_dac_unique[df_dac_unique.LBTESTCD.isin(unique_dac_cd)]

In [27]:
# dac_df dictionary with unique LBTEST and LBTESTCD
unique_dac_lbtest = dict(zip(df_dac_unique.LBTESTCD,df_dac_unique.LBTEST))

In [28]:
# req_code_df dictionary with unique LBTEST and LBTESTCD
unique_req_lbtest = dict(zip(unique_df_code.LBTESTCD,unique_df_code.LBTEST))

In [120]:
ndf = pd.DataFrame(columns = ['LBTESTCD-80','LBTEST-80','a_LBTESTCD-80','a_LBTEST-80'])

In [122]:
i = 0
for k,v in unique_req_lbtest.items():
    for k1,v1 in unique_dac_lbtest.items():
        
        if difflib.SequenceMatcher(None,v,v1).ratio() >=0.8 :
            ndf.loc[i] =[k,v,k1,v1]
            print ndf.loc[i]
            i += 1
           
            #print v,'\t\t',v1,'\t\t',k,'\t\t',k1

LBTESTCD-80                                HBAB
LBTEST-80            Hepatitis B Virus Antibody
a_LBTESTCD-80                           HAABIGM
a_LBTEST-80      Hepatitis A Virus Antibody IgM
Name: 0, dtype: object
LBTESTCD-80                              EBCIGMAB
LBTEST-80        Epstein-Barr Capsid IgM Antibody
a_LBTESTCD-80                            EBCIGMAG
a_LBTEST-80       Epstein-Barr Capsid IgM Antigen
Name: 1, dtype: object
LBTESTCD-80                                HEAB
LBTEST-80            Hepatitis E Virus Antibody
a_LBTESTCD-80                           HAABIGM
a_LBTEST-80      Hepatitis A Virus Antibody IgM
Name: 2, dtype: object
LBTESTCD-80                    UREAN
LBTEST-80              Urea Nitrogen
a_LBTESTCD-80                    BUN
a_LBTEST-80      Blood Urea Nitrogen
Name: 3, dtype: object
LBTESTCD-80                               HBEAB
LBTEST-80          Hepatitis B Virus e Antibody
a_LBTESTCD-80                           HAABIGM
a_LBTEST-80      Hepatitis A Vir

In [123]:
 ndf

,LBTESTCD-80,LBTEST-80,a_LBTESTCD-80,a_LBTEST-80
0,HBAB,Hepatitis B Virus Antibody,HAABIGM,Hepatitis A Virus Antibody IgM
1,EBCIGMAB,Epstein-Barr Capsid IgM Antibody,EBCIGMAG,Epstein-Barr Capsid IgM Antigen
2,HEAB,Hepatitis E Virus Antibody,HAABIGM,Hepatitis A Virus Antibody IgM
3,UREAN,Urea Nitrogen,BUN,Blood Urea Nitrogen
4,HBEAB,Hepatitis B Virus e Antibody,HAABIGM,Hepatitis A Virus Antibody IgM
5,HAVVLD,HAV Viral Load,HHV6VLD,HHV-6 Viral Load
6,HCAG,Hepatitis C Virus Antigen,HAABIGM,Hepatitis A Virus Antibody IgM
7,HIVVLD,HIV Viral Load,HHV6VLD,HHV-6 Viral Load
8,HDAB,Hepatitis D Virus Antibody,HAABIGM,Hepatitis A Virus Antibody IgM
9,EBVVLD,EBV Viral Load,HHV6VLD,HHV-6 Viral Load


In [124]:
ndf2 = pd.DataFrame(columns = ['LBTESTCD80-50','LBTEST80-50','a_LBTESTCD80-50','a_LBTEST80-50'])
i = 0
for k,v in unique_req_lbtest.items():
    for k1,v1 in unique_dac_lbtest.items():
        
        if 0.8 > difflib.SequenceMatcher(None,v,v1).ratio() >= 0.5 :
            ndf2.loc[i] =[k,v,k1,v1]
            i += 1
           
            #print v,'\t\t',v1,'\t\t',k,'\t\t',k1

In [125]:
ndf2

,LBTESTCD80-50,LBTEST80-50,a_LBTESTCD80-50,a_LBTEST80-50
0,HBVVLD,HBV Viral Load (IU/mL),HEVVLD,Hepatitis E Viral Load
1,HBVVLD,HBV Viral Load (IU/mL),PB19VLD,Parvovirus B19 Viral Load
2,HBVVLD,HBV Viral Load (IU/mL),HHV6VLD,HHV-6 Viral Load
3,HBAB,Hepatitis B Virus Antibody,HEVVLD,Hepatitis E Viral Load
4,HBAB,Hepatitis B Virus Antibody,HERNA,Hepatitis E RNA
5,LYMPHOTC,Lymphotactin,LYMLE,Lymphocytes/Leukocytes
6,LYMPHOTC,Lymphotactin,LYMAT,Lymphocytes Atypical
7,HIVAB,HIV Antibody,HAABIGM,Hepatitis A Virus Antibody IgM
8,EBCIGMAB,Epstein-Barr Capsid IgM Antibody,EBN1IGAB,Epstein-Barr Nuclear Antigen 1 IgG Ab
9,SMUSCGAB,Smooth Muscle IgG Antibody,SLAGAB,Soluble Liver Antigen Antibody


In [126]:
ndf3 = pd.DataFrame(columns = ['LBTESTCD - below50','LBTEST- below50','a_LBTESTCD- below50','a_LBTEST- below50'])
i = 0
for k,v in unique_req_lbtest.items():
    for k1,v1 in unique_dac_lbtest.items():
        
        if difflib.SequenceMatcher(None,v,v1).ratio() < 0.5 :
            ndf3.loc[i] =[k,v,k1,v1]
            i += 1
           
            #print v,'\t\t',v1,'\t\t',k,'\t\t',k1

In [127]:
ndf3

,LBTESTCD - below50,LBTEST- below50,a_LBTESTCD- below50,a_LBTEST- below50
0,HBVVLD,HBV Viral Load (IU/mL),KETONES,Ketones
1,HBVVLD,HBV Viral Load (IU/mL),NEUTLE,Neutrophils/Leukocytes
2,HBVVLD,HBV Viral Load (IU/mL),SCSPMSR1,SPE M-spike Reg.1
3,HBVVLD,HBV Viral Load (IU/mL),BACT,Bacteria
4,HBVVLD,HBV Viral Load (IU/mL),SCSPMSR2,SPE M-spike Reg.2
5,HBVVLD,HBV Viral Load (IU/mL),FATBODOV,"Fat Bodies, Oval"
6,HBVVLD,HBV Viral Load (IU/mL),CYTRPHOS,Triple Phosphate Crystals
7,HBVVLD,HBV Viral Load (IU/mL),CYCAOXA,Calcium Oxalate Crystals
8,HBVVLD,HBV Viral Load (IU/mL),SODIUM,Sodium
9,HBVVLD,HBV Viral Load (IU/mL),GLOBG,Gamma Globulin


In [128]:
result = pd.concat([ndf,ndf2,ndf3], axis=1)

In [129]:
result

,LBTESTCD-80,LBTEST-80,a_LBTESTCD-80,a_LBTEST-80,LBTESTCD80-50,LBTEST80-50,a_LBTESTCD80-50,a_LBTEST80-50,LBTESTCD - below50,LBTEST- below50,a_LBTESTCD- below50,a_LBTEST- below50
0,HBAB,Hepatitis B Virus Antibody,HAABIGM,Hepatitis A Virus Antibody IgM,HBVVLD,HBV Viral Load (IU/mL),HEVVLD,Hepatitis E Viral Load,HBVVLD,HBV Viral Load (IU/mL),KETONES,Ketones
1,EBCIGMAB,Epstein-Barr Capsid IgM Antibody,EBCIGMAG,Epstein-Barr Capsid IgM Antigen,HBVVLD,HBV Viral Load (IU/mL),PB19VLD,Parvovirus B19 Viral Load,HBVVLD,HBV Viral Load (IU/mL),NEUTLE,Neutrophils/Leukocytes
2,HEAB,Hepatitis E Virus Antibody,HAABIGM,Hepatitis A Virus Antibody IgM,HBVVLD,HBV Viral Load (IU/mL),HHV6VLD,HHV-6 Viral Load,HBVVLD,HBV Viral Load (IU/mL),SCSPMSR1,SPE M-spike Reg.1
3,UREAN,Urea Nitrogen,BUN,Blood Urea Nitrogen,HBAB,Hepatitis B Virus Antibody,HEVVLD,Hepatitis E Viral Load,HBVVLD,HBV Viral Load (IU/mL),BACT,Bacteria
4,HBEAB,Hepatitis B Virus e Antibody,HAABIGM,Hepatitis A Virus Antibody IgM,HBAB,Hepatitis B Virus Antibody,HERNA,Hepatitis E RNA,HBVVLD,HBV Viral Load (IU/mL),SCSPMSR2,SPE M-spike Reg.2
5,HAVVLD,HAV Viral Load,HHV6VLD,HHV-6 Viral Load,LYMPHOTC,Lymphotactin,LYMLE,Lymphocytes/Leukocytes,HBVVLD,HBV Viral Load (IU/mL),FATBODOV,"Fat Bodies, Oval"
6,HCAG,Hepatitis C Virus Antigen,HAABIGM,Hepatitis A Virus Antibody IgM,LYMPHOTC,Lymphotactin,LYMAT,Lymphocytes Atypical,HBVVLD,HBV Viral Load (IU/mL),CYTRPHOS,Triple Phosphate Crystals
7,HIVVLD,HIV Viral Load,HHV6VLD,HHV-6 Viral Load,HIVAB,HIV Antibody,HAABIGM,Hepatitis A Virus Antibody IgM,HBVVLD,HBV Viral Load (IU/mL),CYCAOXA,Calcium Oxalate Crystals
8,HDAB,Hepatitis D Virus Antibody,HAABIGM,Hepatitis A Virus Antibody IgM,EBCIGMAB,Epstein-Barr Capsid IgM Antibody,EBN1IGAB,Epstein-Barr Nuclear Antigen 1 IgG Ab,HBVVLD,HBV Viral Load (IU/mL),SODIUM,Sodium
9,EBVVLD,EBV Viral Load,HHV6VLD,HHV-6 Viral Load,SMUSCGAB,Smooth Muscle IgG Antibody,SLAGAB,Soluble Liver Antigen Antibody,HBVVLD,HBV Viral Load (IU/mL),GLOBG,Gamma Globulin


In [131]:
result.to_csv("C:/Users/Md.Shamsuzzaman/Documents/OCS/AIH/Daclizumab/similar LBTESTCD.csv", index = False)

In [83]:
for k,v in unique_req_lbtest.items():
    print v,'\t\t','\t\t',k

HBV Viral Load (IU/mL) 				HBVVLD
Hepatitis B Virus Antibody 				HBAB
Lymphotactin 				LYMPHOTC
HCV Genotype 				HCVGTYPE
HIV Antibody 				HIVAB
Epstein-Barr Capsid IgM Antibody 				EBCIGMAB
Smooth Muscle IgG Antibody 				SMUSCGAB
Interferon Beta 				IFNB

HIV-1/2 Antibody 				HIV12AB
Varicella Zoster Virus Antibody 				VZVAB
Hepatitis E Virus Antibody 				HEAB
Varicella Zoster Virus IgA Antibody 				VZVIGAAB
Urea Nitrogen 				UREAN
HBV Genotype 				HBVGTYPE
Hepatitis B Virus e Antigen 				HBEAG
Segmented Neutrophils/Neutrophils 				NEUTSGNE
HBV DNA PCR Taq Man 				HBDTQM
Hepatitis B Virus e Antibody 				HBEAB
Hepatitis B Virus Core IgG Antibody 				HBCIGGAB
HCV RNA Taq 2.0-Exp 				HCRTQE
Fibrosis Score 				FBTSTSCR
HAV Viral Load 				HAVVLD
Prothrombin Intl. Normalized Ratio 				INR
Hepatitis C Virus Antigen 				HCAG
HIV Viral Load 				HIVVLD
Liver Kidney Microsomal Type 1 IgA Ab 				LKM1IAAB
Hepatitis D Virus Antibody 				HDAB
Monoclonal Protein Spike 				MSPIKE
Anti-Neutrophil Cytop

In [85]:
for k1,v1 in unique_dac_lbtest.items():
        
    print v1,'\t\t',k1

Ketones 		KETONES
Neutrophils/Leukocytes 		NEUTLE
SPE M-spike Reg.1 		SCSPMSR1
Bacteria 		BACT
SPE M-spike Reg.2 		SCSPMSR2
Fat Bodies, Oval 		FATBODOV
Triple Phosphate Crystals 		CYTRPHOS
Calcium Oxalate Crystals 		CYCAOXA
Sodium 		SODIUM
Gamma Globulin 		GLOBG
Monocytes 		MONO
Beta Globulin/Total Protein 		GLOBBPT
Beta Globulin 		GLOBB
Phenothiazine 		PHENTHZ
Calcium Carbonate Crystals 		CYCACAR
HSV 1/2 IgM Antibody Interpretation 		HS12GMAI
Waxy Casts 		CSWAX
Alpha-2 Globulin/Total Protein 		GLOBA2PT
Tubular Epithelial Cells 		EPITUCE
Hepatitis E Viral Load 		HEVVLD
Alpha-1 Globulin/Total Protein 		GLOBA1PT
Erythrocyte Cell Clumps 		RBCCLMP
Total Yield 		GTTYIELD
Neutrophils Band Form/Leukocytes 		NEUTBLE
Aliquot 1 Yield 		GTYIELD1
Aliquot 2 Yield 		GTYIELD2
Transitional Epithelial Cells 		EPITCE
Neutrophils Band Form 		NEUTB
Calcium Phosphate Crystals 		CYCAPHOS
Soluble Liver Antigen Antibody 		SLAGAB
Squamous Epithelial Cells 		EPISQCE
Parvovirus B19 Viral Load 		PB19VLD
Methaqual

In [ ]:
# Dict to replace the LBTESTCD with
matched_LBTESTCD = {'UREAN':'BUN', 'HAIGMAB':'HAABIGM', 'ACTMN':'APAP','HIGMIN':'HS12GMAI', 'SLAIGGAB':'SLAGAB','HEVRNA':'HERNA'}

In [ ]:
# Replace non SDTM LBTSTCDs with SDTM LBTSTCDs
for k, v in matched_LBTESTCD.items():
    lb_df.loc[lb_df.LBTESTCD == v] = k

In [ ]:
lb_df.to_csv("C:/Users/Md.Shamsuzzaman/Documents/OCS/AIH/Daclizumab/205MS301_lb.csv",index = False)

In [ ]:
lb_df1 = lb_df

In [6]:
# Importing phase 2 LB data into pandas data frame
df_dac = pd.read_sas(r'\\cdsesub1\\evsprod\\BLA761029\\0014\\m5\\datasets\\205MS201\\tabulations\\sdtm\\lb.xpt')

In [36]:
df_dac.iloc[:,:15].head()

,STUDYID,DOMAIN,USUBJID,LBSEQ,LBREFID,LBTESTCD,LBTEST,LBCAT,LBSCAT,LBORRES,LBORRESU,LBORNRLO,LBORNRHI,LBSTRESC,LBSTRESN
0,205MS201,LB,205MS201/100-001,1.0,L721053,ALP,Alkaline Phosphatase,BLOOD CHEMISTRY,,62,U/L,31,106,62,62.0
1,205MS201,LB,205MS201/100-001,2.0,L721053,ALT,Alanine Aminotransferase,BLOOD CHEMISTRY,,18,U/L,6,34,18,18.0
2,205MS201,LB,205MS201/100-001,3.0,L721053,AST,Aspartate Aminotransferase,BLOOD CHEMISTRY,,14,U/L,9,34,14,14.0
3,205MS201,LB,205MS201/100-001,4.0,L721053,BICARB,Bicarbonate,BLOOD CHEMISTRY,,25.1,mmol/L,17,30.6,25.1,25.1
4,205MS201,LB,205MS201/100-001,5.0,L721053,BUN,Blood Urea Nitrogen,BLOOD CHEMISTRY,,3.0,mmol/L,1.4,8.6,3,3.0


In [41]:
# find common col var 
common_var = list(set(LB_data_elements).intersection(list(df_dac)))

In [42]:
# find the vars which are present in lb_df and missing in df_dac
list(set(LB_data_elements) - set(common_var))

[u'APPLICID', u'LBTOX', u'LBENDTC', u'LBFAST', u'LBSTNRC']

In [43]:
# find the vars which are missing in df_dac
list(set(list(df_dac)) - set(common_var))

[u'LBORRES',
 u'LBORRESU',
 u'LBORNRHI',
 u'LBSCAT',
 u'EPOCH',
 u'LBREFID',
 u'LBNAM',
 u'LBORNRLO',
 u'LBNRIND']

In [44]:
# fill up the dm_df with daclizumab data
lb_df['APPLICID'] = 'BLA761029'
lb_df[common_var] = df_dac[common_var]

In [45]:
# Check the LBTEST code values
dac_cd = set(list(lb_df['LBTESTCD']))

In [46]:
# find common col var 
common_var = list(set(req_code).intersection(list(dac_cd)))

In [47]:
# find the code which are missing in dac_cd
unique_req_cd = list(set(req_code) - set(common_var))

In [50]:
unique_req_cd

[u'HBVVLD',
 u'LKM1IGAB',
 u'BARB',
 u'CMVIGMAB',
 u'HBVGTYPE',
 u'NEUTSGNE',
 u'FBTSTSCR',
 u'HCAG',
 u'HBDNA',
 u'COCAINE',
 u'HS1IGMAB',
 u'HS12GMAB',
 u'CREATCLR',
 u'HEIGGAB',
 u'TRCYANDP',
 u'HDIGMAB',
 u'HIV12AB',
 u'EBCIGGAB',
 u'EBVINTP',
 u'CRLPLSMN',
 u'PCP',
 u'HIVAB',
 u'EBCIGMAB',
 u'HS1IGGAB',
 u'VZVAB',
 u'HEAB',
 u'UREAN',
 u'HIGMIN',
 u'MSPIKE',
 u'AMA',
 u'HDIGGAB',
 u'PROPOX',
 u'HDAB',
 u'CMVIGGAB',
 u'HS2IGMAB',
 u'FBTSTSTG',
 u'HS6AB',
 u'HCVVLD',
 u'IGG',
 u'HCRTQE',
 u'IGE',
 u'IGA',
 u'IGM',
 u'LIPASET',
 u'HBDTQM',
 u'SLAIGGAB',
 u'OPIATE',
 u'HS12GGAB',
 u'EOSCE',
 u'HBCIGMAB',
 u'HBVDNA',
 u'BNZDZPN',
 u'HCVGTYPE',
 u'IFNB\n',
 u'HAAB',
 u'HS2IGGAB',
 u'VZVIGMAB',
 u'HBCIGGAB',
 u'CANNAB',
 u'HIVVLD',
 u'ANCAB',
 u'HBSAB',
 u'HIV2AB',
 u'CMVVLD',
 u'PB19GMAB',
 u'LKM1IMAB',
 u'ANA',
 u'LYMA',
 u'METHDN',
 u'VZVIGGAB',
 u'HIGMC',
 u'HDVVLD',
 u'AMPHET',
 u'BASOCE',
 u'SMUSCGAB',
 u'INR',
 u'HBEAG',
 u'HBEAB',
 u'HEIGMAB',
 u'HBAB',
 u'HAVVLD',
 u'HAIGMAB',
 

In [29]:
# find the code which are missing in req_code
unique_dac_cd = list(set(dac_cd) - set(common_var))

In [30]:
df_dac_unique = df_dac.drop_duplicates(['LBTESTCD'], keep = 'first')

In [31]:
# req_code df with unique req code 
unique_df_code = df_code[df_code.LBTESTCD.isin(unique_req_cd)]

In [32]:
#dac_df with unique dac_code 
df_dac_unique = df_dac_unique[df_dac_unique.LBTESTCD.isin(unique_dac_cd)]

In [33]:
# dac_df dictionary with unique LBTEST and LBTESTCD
unique_dac_lbtest = dict(zip(df_dac_unique.LBTESTCD,df_dac_unique.LBTEST))

In [34]:
# req_code_df dictionary with unique LBTEST and LBTESTCD
unique_req_lbtest = dict(zip(unique_df_code.LBTESTCD,unique_df_code.LBTEST))

In [ ]:
for k,v in unique_req_lbtest.items():
    for k1,v1 in unique_dac_lbtest.items():
        if difflib.SequenceMatcher(None,v,v1).ratio() >=0.7 :
            print v,'\t\t',v1,'\t\t',k,'\t\t',k1

In [ ]:
# Dict to replace the LBTESTCD with
matched_LBTESTCD = {'UREAN':'BUN', 'LIPASET':'LIPASE'}

In [ ]:
# Replace non SDTM LBTSTCDs with SDTM LBTSTCDs
for k, v in matched_LBTESTCD.items():
    lb_df.loc[lb_df.LBTESTCD == v] = k

In [ ]:
lb_df.to_csv("C:/Users/Md.Shamsuzzaman/Documents/OCS/AIH/Daclizumab/205MS201_lb.csv",index = False)

In [ ]:
unique_dac_lbtest = {}

for val in unique_dac_cd:
    unique_dac_lbtest[val] = df_dac_unique[df_dac_unique.LBTESTCD == val].LBTEST

In [ ]:
unique_req_lbtest = {}
for val in unique_req_cd:
    unique_req_lbtest[val] = df_code.ix[df_code.LBTESTCD == val, 'LBTEST'].values
    #print df_code.ix[df_code.LBTESTCD == val, 'LBTEST'].value
#d = dict((df_code[df_code.LBTESTCD == x].LBTEST,x) 

In [ ]:
len(set(unique_dac_lbtest))

In [ ]:
df_dac_unique.head()

In [ ]:
df_dac_unique

In [ ]:
# find common col var 
common_var = list(set(DM_data_elements).intersection(list(df_tenofovir)))

In [ ]:
#how many are common
len(common_var)

In [ ]:
# extract the cocmmon variable data from tenofovir data frame
ndf4 =df_tenofovir[common_var]

In [ ]:
len(ndf2)

In [ ]:
len(ndf4)

In [ ]:
len(ndf3)

In [ ]:
len(ndf1)

In [ ]:
len(ndf)

In [ ]:
# Importing data from daclizumab into pandas data frame
df_daclizumab = pandas.read_sas(r"\\cdsesub1\evsprod\BLA761029\0000\m5\datasets\205ms301\tabulations\sdtm\dm.xpt")

In [ ]:
df_daclizumab.info()

In [ ]:
df_daclizumab.head()

In [ ]:
# find common col var 
common_var = list(set(DM_data_elements).intersection(list(df_daclizumab)))

In [ ]:
#how many are common
len(common_var)

In [ ]:
# Not all 20 are present in daclizumab DM domain. Hence, create an empty df with variables needed first

ddf = pd.DataFrame(index = range(len(df_daclizumab)), columns = DM_data_elements)

In [ ]:
# extract the cocmmon variable data from tenofovir data frame
ddf =df_daclizumab[common_var]

In [ ]:
# merge the tenofovir and daclizumab dfs
fdf = ndf.append(ddf).reset_index()

In [ ]:
fdf.info()

In [ ]:
fdf = fdf.drop('index',1)

In [ ]:
fdf.info()

In [ ]:
fdf.to_csv("extracted_dataset.csv", encoding='utf-8')